In [3]:
import pickle
import pandas as pd

ratings = pickle.load( open( "saveKidsAndAdults.p", "rb" ) )


In [4]:
ratingsPD = pd.DataFrame(ratings, columns=['user', 'item', 'rating'] )
ratingsPD.head()

user  item  rating
0     0    45    33.0
1     0    53     3.0
2     0    78   128.0
3     0   249     4.0
4     0   283    12.0

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = preprocessing.MinMaxScaler([1,5])
def fx(x):
    data = preprocessing.scale(x['rating'].to_numpy().reshape(-1, 1))
    scaler.fit(data)
    data=scaler.transform(data)
    x['rating']=data
    return x
ratingsPD = ratingsPD.groupby('user').apply(fx)

In [6]:
ratingsPD1= ratingsPD.groupby('user').filter(lambda x : len(x)>24)


In [7]:
ratingsPD2 = ratingsPD1.groupby('item').filter(lambda x : len(x)>3)

In [8]:
ratings =ratingsPD2

In [9]:
len(ratings)

16361021

In [10]:
ratings.groupby('user').count()

item  rating
user               
0       896     896
1      1959    1959
2      2409    2409
3       803     803
4       482     482
5      1455    1455
6       241     241
7       922     922
8      1802    1802
9       218     218
10     1132    1132
11      851     851
12      679     679
13      927     927
14       98      98
15      194     194
16     2051    2051
17      361     361
18      365     365
19      788     788
20     2215    2215
21      939     939
22      635     635
23     2541    2541
24      465     465
25     3978    3978
26     1260    1260
27      189     189
28     1651    1651
29      988     988
...     ...     ...
28299   123     123
28300   128     128
28302   268     268
28303    45      45
28304    30      30
28305    49      49
28307   171     171
28308    75      75
28309    90      90
28310    30      30
28311    84      84
28312   107     107
28313    25      25
28314    30      30
28315    54      54
28316   173     173
28317   246     246
28318    67      67
28319    98      98
28320   109     109
28323    25      25
28324    35      35
28325    92      92
28326   526     526
28328   153     153
28329    66      66
28330    57      57
28332    39      39
28333    26      26
28334   120     120

[27985 rows x 2 columns]

In [11]:
ratings['item'].nunique()

307774

In [12]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.BiasedMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))


0.34382157424787063
0.34592133139168363
0.341455067348996
0.34569089353870863
0.34693531843668374
Average RMSE: 0.34476483699278854


In [ ]:
import lenskit
from lenskit.algorithms.basic import Bias
from lenskit.metrics.predict import rmse

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn, user_knn as uknn, funksvd
from lenskit.algorithms.basic import Fallback
from lenskit import topn
from lenskit.algorithms.basic import Bias
rm = 0
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5,xf.SampleFrac(0.3)):
    pred = als.ImplicitMF(45)
    #rec = Recommender.adapt(pred)
    pred.fit(train)
    preds = batch.predict(pred, test)
    #print(preds['prediction'].isna().sum())
    print((rmse(preds['prediction'], preds['rating'],missing ='ignore')))
    rm+= (rmse(preds['prediction'], preds['rating'],missing ='ignore'))
print("Average RMSE: " + str(rm/5))